In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from math import sqrt, pi, exp
import pandas as pd
import missingno as msno
from math import *
from scipy import stats
from pydoc import help
from sklearn import preprocessing
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn import datasets
from sklearn import svm
from sklearn.model_selection import ShuffleSplit
import glob, os
import seaborn as sns
import pickle
from sklearn.feature_extraction import DictVectorizer
from nltk.tokenize import RegexpTokenizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import f1_score,precision_score,recall_score
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import Perceptron
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import model_selection
import xgboost as XGB
from sklearn.model_selection import cross_validate

In [ ]:
X_train = '/content/drive/MyDrive/Colab_Notebooks/P6/FinalTrain10K.pkl'
with open(X_train, 'rb') as file6:
    X_train = pickle.load(file6)

# Récupération de la matrice
X_test = '/content/drive/MyDrive/Colab_Notebooks/P6/FinalTest10K.pkl'
with open(X_test, 'rb') as file6:
    X_test = pickle.load(file6)

X_train_vectorized = '/content/drive/MyDrive/Colab_Notebooks/P6/FinalTrainVectorized10K.pkl'
with open(X_train_vectorized, 'rb') as file6:
    X_train_vectorized = pickle.load(file6)

# Récupération de la matrice
X_test_vectorized = '/content/drive/MyDrive/Colab_Notebooks/P6/FinalTestVectorized10K.pkl'
with open(X_test_vectorized, 'rb') as file6:
    X_test_vectorized = pickle.load(file6)
    
# Récupération de la matrice
y_train = '/content/drive/MyDrive/Colab_Notebooks/P6/FinalTrainLabels10K.pkl'
with open(y_train, 'rb') as file6:
    y_train = pickle.load(file6)

# Récupération de la matrice
y_test = '/content/drive/MyDrive/Colab_Notebooks/P6/FinalTestLabels10K.pkl'
with open(y_test, 'rb') as file6:
    y_test = pickle.load(file6)

# Récupération de la matrice
mlb = '/content/drive/MyDrive/Colab_Notebooks/P6/mlb10K.pkl'
with open(mlb, 'rb') as file6:
    mlb = pickle.load(file6)

In [ ]:
print(X_train.shape, X_test.shape, X_train_vectorized.shape, X_test_vectorized.shape, y_train.shape, y_test.shape)

(8000, 30) (2000, 30) (8000, 1312) (2000, 1312) (8000, 30) (2000, 30)


#### 1) Test modèles

In [ ]:
%%time
results = []
names = []
models = []

LR_model = LogisticRegression(max_iter=15000)
RFC_model = RandomForestClassifier()
svc = SVC()
XGB_model = XGB.XGBClassifier()

models.append(('XGB', OneVsRestClassifier(XGB_model)))
models.append(('RFC', OneVsRestClassifier(RFC_model)))
models.append(('LR', OneVsRestClassifier(LR_model)))
scoring = ['jaccard_weighted', 'f1_weighted']
for name, model in models:
    cv_results = cross_validate(model, X_train_vectorized, y_train, cv = 3,
                           scoring = scoring,
                           return_train_score = True,n_jobs=-1)
    results.append(cv_results)
    names.append(name)
    result_df = pd.DataFrame(results, names)
    for i in result_df.columns:
        if i != 'model':
            result_df[i] = result_df[i].map(lambda x: x.mean())

CPU times: user 10 s, sys: 3.25 s, total: 13.3 s
Wall time: 38min 44s


In [ ]:
result_df

,fit_time,score_time,test_jaccard_weighted,train_jaccard_weighted,test_f1_weighted,train_f1_weighted
XGB,1219.396193,1.310243,0.371718,0.827678,0.518626,0.898682
RFC,1066.621733,2.375880,0.124669,0.999308,0.214376,0.999653
LR,9.942465,0.163242,0.272065,0.327180,0.402397,0.465333


In [ ]:
LR_model = LogisticRegression(max_iter=15000)
clf = OneVsRestClassifier(LR_model)
clf.fit(X_train_vectorized, y_train)


y_pred = clf.predict(X_test_vectorized)

In [ ]:
with open('/content/drive/MyDrive/Colab_Notebooks/P6/Modele10K.pkl','wb') as f:
    pickle.dump(clf, f)

#### 1.1) Application du modèle de classification sur le test pour l'API (test des tags renvoyés)

In [ ]:
LR_model = LogisticRegression(max_iter=15000)
clf = OneVsRestClassifier(LR_model)
clf.fit(X_train_vectorized, y_train)

OneVsRestClassifier(estimator=LogisticRegression(C=1.0, class_weight=None,
                                                 dual=False, fit_intercept=True,
                                                 intercept_scaling=1,
                                                 l1_ratio=None, max_iter=15000,
                                                 multi_class='auto',
                                                 n_jobs=None, penalty='l2',
                                                 random_state=None,
                                                 solver='lbfgs', tol=0.0001,
                                                 verbose=0, warm_start=False),
                    n_jobs=None)

In [ ]:
y_pred = clf.predict(X_test_vectorized)

In [ ]:
lTag = mlb.inverse_transform(y_pred)
Tags_predicted = list(lTag)
Tags_predicted

[(),
 ('javascript',),
 (),
 (),
 (),
 ('c#',),
 (),
 (),
 (),
 ('linux',),
 (),
 (),
 ('json',),
 (),
 (),
 ('python',),
 (),
 (),
 (),
 (),
 ('python',),
 (),
 (),
 (),
 ('android',),
 ('python',),
 ('css',),
 ('python',),
 (),
 (),
 (),
 (),
 (),
 ('swift',),
 (),
 (),
 (),
 ('java',),
 ('python',),
 (),
 ('javascript',),
 (),
 (),
 ('javascript',),
 ('android',),
 (),
 ('html', 'css'),
 (),
 (),
 (),
 ('javascript',),
 ('ios',),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 ('java',),
 (),
 (),
 (),
 ('html', 'css'),
 (),
 ('python',),
 (),
 ('c', 'arrays'),
 ('python', 'pandas'),
 (),
 (),
 (),
 (),
 (),
 ('html', 'css'),
 (),
 (),
 (),
 (),
 (),
 (),
 ('java',),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 ('firebase',),
 (),
 (),
 ('r',),
 (),
 (),
 (),
 ('python',),
 (),
 ('c#',),
 ('c++',),
 ('java',),
 (),
 (),
 ('javascript',),
 (),
 (),
 ('java', 'android'),
 (),
 ('html',),
 (),
 ('regex',),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 (),
 ('javascript',),
 ('javascript',),
 (),
 ('python',),
 

In [ ]:
X_test['Concat_body_title_cleaned'].iloc[10]

'code typedef template typename boost variant int string double bool member mean mean'

In [ ]:
X_test['Tags'].iloc[10]

'c++ template-meta-programming '

In [ ]:
y_test.iloc[10]

Tag_python          0
Tag_java            0
Tag_javascript      0
Tag_c#              0
Tag_html            0
Tag_android         0
Tag_c++             1
Tag_css             0
Tag_php             0
Tag_r               0
Tag_ios             0
Tag_swift           0
Tag_c               0
Tag_arrays          0
Tag_jquery          0
Tag_python-3.x      0
Tag_node.js         0
Tag_mysql           0
Tag_pandas          0
Tag_bash            0
Tag_json            0
Tag_angular         0
Tag_linux           0
Tag_reactjs         0
Tag_regex           0
Tag_sql             0
Tag_string          0
Tag_ecmascript-6    0
Tag_.net            0
Tag_firebase        0
Name: 10, dtype: int64

In [ ]:
lTag = mlb.inverse_transform(y_pred)
Tags_predicted = list(lTag[10])
Tags_predicted

[]

#### 2) Optimisation des hyperparamètres

###### 2.1) RandomizedSearchCV sur le RandomForestClassifier (non retenu)

In [ ]:
%%time

from sklearn.model_selection import RandomizedSearchCV, GridSearchCV, train_test_split
from sklearn.linear_model import SGDRegressor
from sklearn.ensemble import RandomForestClassifier

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 500, num = 5)] # à varier après avoir fixer les autres hyperparamètres # boucle avec les autres hyperparamètres et faire varier le n_estimators 100 200 300 400 graph pour montrer lorsque l'amélioration s'arrête
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 40, num = 4)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
               
# First create the base model to tune
rf = RandomForestClassifier()    

rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 10, cv = 3, verbose=2, random_state=42, n_jobs = -1)

CPU times: user 315 µs, sys: 0 ns, total: 315 µs
Wall time: 323 µs


In [ ]:
%%time

# Fit the random search model
rf_random.fit(X_train_vectorized,y_train)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 40 concurrent workers.
[Parallel(n_jobs=-1)]: Done  15 out of  30 | elapsed:  4.1min remaining:  4.1min
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:  9.6min finished


KeyboardInterrupt: ignored

In [ ]:
rf_random.best_params_

In [ ]:
%%time

from sklearn.model_selection import RandomizedSearchCV, GridSearchCV, train_test_split
from sklearn.linear_model import SGDRegressor
from sklearn.ensemble import RandomForestClassifier

# Number of trees in random forest
n_estimators = [200] # à varier après avoir fixer les autres hyperparamètres # boucle avec les autres hyperparamètres et faire varier le n_estimators 100 200 300 400 graph pour montrer lorsque l'amélioration s'arrête
# Number of features to consider at every split
max_features = ['auto']
# Maximum number of levels in tree
max_depth = [40]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1]
# Method of selecting samples for training each tree
bootstrap = [False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
               
# First create the base model to tune
rf = RandomForestClassifier()    


rf.fit(X_train_vectorized,y_train)

CPU times: user 5min 41s, sys: 686 ms, total: 5min 42s
Wall time: 5min 41s


######2.2) RandomizedSearchCV sur le XGBoost (non retenu)

In [ ]:
# First create the base model to tune
xgb = XGB.XGBClassifier()  
xgb_random = RandomizedSearchCV(estimator = xgb, param_distributions = params, n_iter = 10, cv = 3, verbose=2, random_state=42, n_jobs = -1)

In [ ]:
%%time
#params = {'estimator__min_child_weight': [2, 4],
#        'estimator__gamma': [0.5, 1],
#        'estimator__subsample': [0.5, 1],
#        'estimator__colsample_bytree': [0.5, 1],
#        'estimator__max_depth': [2, 4]
#        }

#xgb_2 = OneVsRestClassifier(xgb)
#grid_xgb = GridSearchCV(estimator = xgb_2, param_grid = params, cv = 5,n_jobs = -1)
#grid_xgb.fit(X_train_vectorized, y_train)

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 6.44 µs


In [ ]:
#grid_xgb.best_params_

######2.3) GridsearchCV sur le Logistic Regression (retenu)

In [ ]:
%%time
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV, train_test_split
lr_params = {'estimator__C' : [100, 10, 1.0],
          'estimator__penalty' : ['l1', 'l2'],
          }

lr_2 = OneVsRestClassifier(LogisticRegression())
lr_clf = GridSearchCV(estimator = lr_2, param_grid = lr_params, cv = 3, n_jobs = -1)
grid_lr = lr_clf.fit(X_train_vectorized, y_train)  

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


CPU times: user 5min 52s, sys: 5min 18s, total: 11min 11s
Wall time: 53.7 s


In [ ]:
grid_lr.cv_results_

{'mean_fit_time': array([ 0.05346982, 34.92360139,  0.05657713, 23.96053251,  0.05338454,
        12.2339824 ]),
 'mean_score_time': array([0.        , 0.1390756 , 0.        , 0.1588432 , 0.        ,
        0.18946012]),
 'mean_test_score': array([       nan, 0.48049845,        nan, 0.48024834,        nan,
        0.34724828]),
 'param_estimator__C': masked_array(data=[100, 100, 10, 10, 1.0, 1.0],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_estimator__penalty': masked_array(data=['l1', 'l2', 'l1', 'l2', 'l1', 'l2'],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'estimator__C': 100, 'estimator__penalty': 'l1'},
  {'estimator__C': 100, 'estimator__penalty': 'l2'},
  {'estimator__C': 10, 'estimator__penalty': 'l1'},
  {'estimator__C': 10, 'estimator__penalty': 'l2'},
  {'estimator__C': 1.0, 'estimator__penalty': 'l1'},
  {'estima

In [ ]:
grid_lr.score()

TypeError: ignored

In [ ]:
grid_lr.best_params_

Application des hyperparamètres

In [ ]:
%%time
lr_params = {'estimator__C' : [100],
          'estimator__penalty' : ['l2']
          }

LR_model = LogisticRegression(max_iter=15000,C=100, penalty='l2')
clf = OneVsRestClassifier(LR_model)
clf.fit(X_train_vectorized, y_train)
y_pred = clf.predict(X_test_vectorized)